# Abundance zeropoint analysis

## Author(s): Sven Buder (SB, WG4)

### History:
180926 SB Created

In [2]:
# Preamble for notebook 

# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

# import matplotlib
# matplotlib.use('Agg')
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic packages
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import sys
import collections
import glob
import pickle
import pandas

# Packages to work with FITS and (IDL) SME.out files
import astropy.io.fits as pyfits
import astropy.table as table
from astropy.table import Table
from scipy.io.idl import readsav

# Matplotlib and associated packages for plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.transforms import Bbox,TransformedBbox
from matplotlib.image import BboxImage
from matplotlib.legend_handler import HandlerBase
from matplotlib._png import read_png
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors

params = {
    'font.family'        : 'sans',
    'font.size'          : 17,
    'axes.labelsize'     : 20,
    'ytick.labelsize'    : 16,
    'xtick.labelsize'    : 16,
    'legend.fontsize'    : 20,
    'text.usetex'        : True, 
    'text.latex.preamble': [r'\usepackage{upgreek}', r'\usepackage{amsmath}'],
    }   
plt.rcParams.update(params)

_parula_data = [[0.2081, 0.1663, 0.5292], 
                [0.2116238095, 0.1897809524, 0.5776761905], 
                [0.212252381, 0.2137714286, 0.6269714286], 
                [0.2081, 0.2386, 0.6770857143], 
                [0.1959047619, 0.2644571429, 0.7279], 
                [0.1707285714, 0.2919380952, 0.779247619], 
                [0.1252714286, 0.3242428571, 0.8302714286], 
                [0.0591333333, 0.3598333333, 0.8683333333], 
                [0.0116952381, 0.3875095238, 0.8819571429], 
                [0.0059571429, 0.4086142857, 0.8828428571], 
                [0.0165142857, 0.4266, 0.8786333333], 
                [0.032852381, 0.4430428571, 0.8719571429], 
                [0.0498142857, 0.4585714286, 0.8640571429], 
                [0.0629333333, 0.4736904762, 0.8554380952], 
                [0.0722666667, 0.4886666667, 0.8467], 
                [0.0779428571, 0.5039857143, 0.8383714286], 
                [0.079347619, 0.5200238095, 0.8311809524], 
                [0.0749428571, 0.5375428571, 0.8262714286], 
                [0.0640571429, 0.5569857143, 0.8239571429], 
                [0.0487714286, 0.5772238095, 0.8228285714], 
                [0.0343428571, 0.5965809524, 0.819852381], 
                [0.0265, 0.6137, 0.8135], 
                [0.0238904762, 0.6286619048, 0.8037619048], 
                [0.0230904762, 0.6417857143, 0.7912666667], 
                [0.0227714286, 0.6534857143, 0.7767571429], 
                [0.0266619048, 0.6641952381, 0.7607190476], 
                [0.0383714286, 0.6742714286, 0.743552381], 
                [0.0589714286, 0.6837571429, 0.7253857143], 
                [0.0843, 0.6928333333, 0.7061666667], 
                [0.1132952381, 0.7015, 0.6858571429], 
                [0.1452714286, 0.7097571429, 0.6646285714], 
                [0.1801333333, 0.7176571429, 0.6424333333], 
                [0.2178285714, 0.7250428571, 0.6192619048], 
                [0.2586428571, 0.7317142857, 0.5954285714], 
                [0.3021714286, 0.7376047619, 0.5711857143], 
                [0.3481666667, 0.7424333333, 0.5472666667], 
                [0.3952571429, 0.7459, 0.5244428571], 
                [0.4420095238, 0.7480809524, 0.5033142857], 
                [0.4871238095, 0.7490619048, 0.4839761905], 
                [0.5300285714, 0.7491142857, 0.4661142857], 
                [0.5708571429, 0.7485190476, 0.4493904762],
                [0.609852381, 0.7473142857, 0.4336857143], 
                [0.6473, 0.7456, 0.4188], 
                [0.6834190476, 0.7434761905, 0.4044333333], 
                [0.7184095238, 0.7411333333, 0.3904761905], 
                [0.7524857143, 0.7384, 0.3768142857], 
                [0.7858428571, 0.7355666667, 0.3632714286], 
                [0.8185047619, 0.7327333333, 0.3497904762], 
                [0.8506571429, 0.7299, 0.3360285714], 
                [0.8824333333, 0.7274333333, 0.3217], 
                [0.9139333333, 0.7257857143, 0.3062761905], 
                [0.9449571429, 0.7261142857, 0.2886428571], 
                [0.9738952381, 0.7313952381, 0.266647619], 
                [0.9937714286, 0.7454571429, 0.240347619], 
                [0.9990428571, 0.7653142857, 0.2164142857], 
                [0.9955333333, 0.7860571429, 0.196652381], 
                [0.988, 0.8066, 0.1793666667], 
                [0.9788571429, 0.8271428571, 0.1633142857], 
                [0.9697, 0.8481380952, 0.147452381], 
                [0.9625857143, 0.8705142857, 0.1309], 
                [0.9588714286, 0.8949, 0.1132428571], 
                [0.9598238095, 0.9218333333, 0.0948380952], 
                [0.9661, 0.9514428571, 0.0755333333], 
                [0.9763, 0.9831, 0.0538]]

parula = ListedColormap(_parula_data, name='parula')
parula_zero = _parula_data[0]
parula_0 = ListedColormap(_parula_data, name='parula_0')
parula_0.set_bad((1,1,1))
parula_r = ListedColormap(_parula_data[::-1], name='parula_r')

willi_blau = [0.0722666667, 0.4886666667, 0.8467]

In [3]:
data = Table.read('../data_products/GALAH_iDR3_random10000.fits',1)

In [4]:
import pandas 
df = pandas.read_csv('../../mode_DR3', delim_whitespace=True, usecols=(0,1,2,3,4,5))

mode = collections.OrderedDict()
mode['element'] = np.array([x for x in df[';elem'][2:]])
mode['lambda']  = np.array([x for x in df['lambda'][2:]])
mode['maskst']  = np.array([x for x in df['maskst'][2:]])
mode['maskend'] = np.array([x for x in df['maskend'][2:]])
mode['segst']   = np.array([x for x in df['segst'][2:]])
mode['segend']  = np.array([x for x in df['segend'][2:]])

In [5]:
from scipy.io import readsav

In [6]:
unique_elements = np.unique(mode['element'])
unique_lines = unique_elements[[len(element) > 4 for element in unique_elements]]

In [7]:
unique_lines = np.array(['Li'])

In [10]:
kwargs_plot = dict(lw=1, alpha=0.03, c='k', rasterized=True)

for each_element in unique_lines:
    
    in_mode = np.where(mode['element'] == each_element)[0]
    
    if len(in_mode) == 1:

        f, (ax1, ax2) = plt.subplots(2, 1, figsize=(15,10))

        for each_sobject_id in data['sobject_id'][:100]:

            # PLOT SME NORMALISED SPECTRUM
            if len(glob.glob('../../OUTPUT_10000_lbol/10000_lbol_'+str(each_sobject_id)+'_DR3_'+each_element+'_SME.out')) > 0:
                sme = readsav('../../OUTPUT_10000_lbol/10000_lbol_'+str(each_sobject_id)+'_DR3_'+each_element+'_SME.out').sme[0]

                ax1.plot(
                    sme.wave,
                    sme.sob,
                    **kwargs_plot
                )
                # We could, but do not plot the synthesised signal
                #ax2.plot(
                #    sme.wave,
                #    sme.smod,
                #    **kwargs_plot
                #)

            # PLOT GUESS NORMALISED SPECTRUM
            spectrum = 0
            if each_element == 'Li':
                if len(glob.glob('../../SPECTRA/dr5.3/'+str(each_sobject_id)[:6]+'/standard/com/'+str(each_sobject_id)+'3.fits')) > 0:
                    spectrum = '../../SPECTRA/dr5.3/'+str(each_sobject_id)[:6]+'/standard/com/'+str(each_sobject_id)+'3.fits'
            elif each_element[2] == '4':
                if len(glob.glob('../../SPECTRA/dr5.3/'+str(each_sobject_id)[:6]+'/standard/com/'+str(each_sobject_id)+'1.fits')) > 0:
                    spectrum = '../../SPECTRA/dr5.3/'+str(each_sobject_id)[:6]+'/standard/com/'+str(each_sobject_id)+'1.fits'
            elif each_element[2] == '5':
                if len(glob.glob('../../SPECTRA/dr5.3/'+str(each_sobject_id)[:6]+'/standard/com/'+str(each_sobject_id)+'2.fits')) > 0:
                    spectrum = '../../SPECTRA/dr5.3/'+str(each_sobject_id)[:6]+'/standard/com/'+str(each_sobject_id)+'2.fits'
            elif each_element[2] == '6':
                if len(glob.glob('../../SPECTRA/dr5.3/'+str(each_sobject_id)[:6]+'/standard/com/'+str(each_sobject_id)+'3.fits')) > 0:
                    spectrum = '../../SPECTRA/dr5.3/'+str(each_sobject_id)[:6]+'/standard/com/'+str(each_sobject_id)+'3.fits'
            elif each_element[2] == '7':
                if len(glob.glob('../../SPECTRA/dr5.3/'+str(each_sobject_id)[:6]+'/standard/com/'+str(each_sobject_id)+'4.fits')) > 0:
                    spectrum = '../../SPECTRA/dr5.3/'+str(each_sobject_id)[:6]+'/standard/com/'+str(each_sobject_id)+'4.fits'
            if spectrum != 0:
                try:
                    fits = pyfits.open(spectrum)
                    start_wavelength = fits[4].header["CRVAL1"]
                    dispersion       = fits[4].header["CDELT1"]
                    nr_pixels        = fits[4].header["NAXIS1"]
                    reference_pixel  = fits[4].header["CRPIX1"]
                    if reference_pixel == 0:
                        reference_pixel = 1
                    wave = np.array(map(lambda x:((x-reference_pixel+1)*dispersion+start_wavelength),range(0,nr_pixels)))
                    sob  = np.array(fits[4].data)

                    in_segment = (wave >= mode['segst'][in_mode] - 3) & (wave <= mode['segend'][in_mode] + 3)
                    ax2.plot(
                        wave[in_segment],
                        sob[in_segment],
                        **kwargs_plot
                    )
                except:
                    pass

        ax1.axvline(mode['lambda'][in_mode],c='r')
        ax1.axvline(mode['maskst'][in_mode],c='y')
        ax1.axvline(mode['maskend'][in_mode],c='y')
        ax1.axvline(mode['segst'][in_mode],c='b')
        ax1.axvline(mode['segend'][in_mode],c='b')
        ax2.axvline(mode['lambda'][in_mode],c='r')
        ax2.axvline(mode['maskst'][in_mode],c='y')
        ax2.axvline(mode['maskend'][in_mode],c='y')
        ax2.axvline(mode['segst'][in_mode],c='b')
        ax2.axvline(mode['segend'][in_mode],c='b')

        ax1.set_xlim(mode['segst'][in_mode] - 3, mode['segend'][in_mode] + 3)
        ax2.set_xlim(mode['segst'][in_mode] - 3, mode['segend'][in_mode] + 3)
        ax1.set_ylim(0,1.2)
        ax2.set_ylim(0,1.2)

        ax1.set_xlabel(r'Wavelength [\AA]')
        ax2.set_xlabel(r'Wavelength [\AA]')

        ax1.set_ylabel('SOB SME')
        ax2.set_ylabel('SOB GUESS')

        ax1.set_title(each_element)
        ax2.set_title(each_element)

        plt.tight_layout()
        try:
            plt.savefig('abundance_masks/SME_random10000_masks_'+each_element+'.pdf',dpi=300,bbox_inches='tight')
        except:
            pass
        plt.close()